In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.optimizer import Optimizer
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from sklearn.metrics import accuracy_score


In [ ]:
embed_size=300
hidden_size=128
max_features=120000
maxlen=700
batch_size=300
n_epochs=10

In [ ]:
url='https://drive.google.com/file/d/1xEyhcHjrjEo62k84kKu6mI9vttGtc-jV/view?usp=share_link'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url)
df=df.drop_duplicates()
df=df[df['Message'].notnull()]
df=df[df['Category'].notnull()]


In [ ]:
print(df.head())

  Category                                            Message
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...


In [ ]:
df=df[pd.notnull(df['Message'])]

In [ ]:
df['len']=df['Message'].apply(lambda s:len(s))
print(df['len'])

0       111
1        29
2       155
3        49
4        61
       ... 
5567    160
5568     36
5569     57
5570    125
5571     26
Name: len, Length: 5157, dtype: int64


In [ ]:
def clean_text(x):
  pattern=r'[^a-zA-z0-9\s]'
  text= re.sub(pattern,'',x)
  return x

In [ ]:
def clean_numbers(x):
  if bool(re.search(r'\d',x)):
    x=re.sub('[0-9]{5,}', '#####', x)
    x=re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
  return x




In [ ]:
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

In [ ]:
def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re
contractions, contractions_re = _get_contractions(contraction_dict)
def replace_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, text)
# Usage
replace_contractions("this's a text with contraction")



'this is a text with contraction'

In [ ]:
# lower the text
df['Message'] = df['Message'].apply(lambda x: x.lower())

# Clean the text
df['Message'] = df['Message'].apply(lambda x: clean_text(x))

# Clean numbers
df['Message'] = df['Message'].apply(lambda x: clean_numbers(x))

# Clean Contractions
df['Message'] = df['Message'].apply(lambda x: replace_contractions(x))


In [ ]:

train_x,test_x,train_y,test_y= train_test_split(df['Message'],df['Category'],stratify=df['Category'],test_size=0.25)

In [ ]:
print('Train shape: ', train_x.shape)
print('Test shape: ', test_x.shape)

Train shape:  (3867,)
Test shape:  (1290,)


In [ ]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_x))
train_x = tokenizer.texts_to_sequences(train_x)
test_x = tokenizer.texts_to_sequences(test_x)


In [ ]:
train_x = pad_sequences(train_x, maxlen=maxlen)
test_x = pad_sequences(test_x, maxlen=maxlen)

In [ ]:
le = LabelEncoder()
train_y = le.fit_transform(train_y.values)
test_y = le.transform(test_y.values)

In [ ]:
le.classes_

array(['ham', 'spam'], dtype=object)

In [ ]:
train_x = torch.tensor(train_x)
train_y = torch.tensor(train_y)
test_x = torch.tensor(test_x)
test_y = torch.tensor(test_y)

In [ ]:
train_datasets = torch.utils.data.TensorDataset(train_x, train_y)
test_datasets = torch.utils.data.TensorDataset(test_x, test_y)

In [ ]:
train_dataLoader = torch.utils.data.DataLoader(train_datasets, batch_size=batch_size, shuffle=True)
test_dataLoader = torch.utils.data.DataLoader(test_datasets, batch_size=batch_size, shuffle=False)


In [ ]:
class MyRNN(nn.Module):
  def __init__(self,embed_size,hidden_size):
    super().__init__()
    self.embedding_layer=nn.Embedding(max_features, embed_size)
    self.rnn=nn.RNN(embed_size,hidden_size,batch_first=True)
  def forward(self,x):
    x=self.embedding_layer(x)
    output,h=self.rnn(x)
    return output,h

In [ ]:
class Attention(nn.Module):
  def __init__(self,hidden_size):
    super().__init__()
    self.attention_score=nn.Linear(hidden_size,1)
    self.sigmoid=nn.Sigmoid()
  def forward(self,x):
    attentions=self.attention_score(x)
    attentions=self.sigmoid(attentions)
    context_vector=torch.sum(attentions*x,dim=1)
    return context_vector

In [ ]:
class MyModel(nn.Module):
  def __init__(self,embed_size,hidden_size,num_class=1):
    super().__init__()
    self.my_rnn=MyRNN(embed_size,hidden_size)
    self.attention_net=Attention(hidden_size)
    self.sigmoid=nn.Sigmoid()
    self.linear_layer=nn.Linear(hidden_size,num_class)
  def forward(self,x):
    output,h=self.my_rnn(x)
    context_vector=self.attention_net(output)
    x=self.linear_layer(context_vector)
    x=self.sigmoid(x)
    return x

In [ ]:
model=MyModel(embed_size,hidden_size)

In [ ]:
criterion=nn.BCELoss(reduction='mean')

In [ ]:
#criterion=nn.CrossEntropyLoss(reduction='sum')

In [ ]:
optimizer=torch.optim.Adam(model.parameters(),lr=0.01)

In [ ]:
for epoch in range(10):
  overall_accuracy=0
  for x,y in train_dataLoader:
    predicted_y=model(x)

    batch_size=x.shape[0]
    y=y.view(batch_size,1)
    predicted_y=torch.tensor(predicted_y,dtype=torch.float)
    y=torch.tensor(y,dtype=torch.float)
    loss=criterion(predicted_y,y)

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()
    y_true=y.detach().numpy()

    y_pred=predicted_y.detach().numpy() >0.5



    accuracy= accuracy_score(y_true,y_pred)

    overall_accuracy +=accuracy*batch_size

  print('\n')
  print(f'Epoch: {epoch} --> Accuracy {overall_accuracy/len(train_datasets)}')
  print('\n')



<ipython-input-30-340c52a80809>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predicted_y=torch.tensor(predicted_y,dtype=torch.float)
<ipython-input-30-340c52a80809>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y=torch.tensor(y,dtype=torch.float)
<ipython-input-30-340c52a80809>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predicted_y=torch.tensor(predicted_y,dtype=torch.float)
<ipython-input-30-340c52a80809>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.c



Epoch: 0 --> Accuracy 0.12438582880786139




<ipython-input-30-340c52a80809>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predicted_y=torch.tensor(predicted_y,dtype=torch.float)
<ipython-input-30-340c52a80809>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y=torch.tensor(y,dtype=torch.float)
<ipython-input-30-340c52a80809>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predicted_y=torch.tensor(predicted_y,dtype=torch.float)
<ipython-input-30-340c52a80809>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.c



Epoch: 1 --> Accuracy 0.12438582880786139




<ipython-input-30-340c52a80809>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predicted_y=torch.tensor(predicted_y,dtype=torch.float)
<ipython-input-30-340c52a80809>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y=torch.tensor(y,dtype=torch.float)
<ipython-input-30-340c52a80809>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predicted_y=torch.tensor(predicted_y,dtype=torch.float)
<ipython-input-30-340c52a80809>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.c

...

<ipython-input-30-340c52a80809>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predicted_y=torch.tensor(predicted_y,dtype=torch.float)
<ipython-input-30-340c52a80809>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y=torch.tensor(y,dtype=torch.float)
<ipython-input-30-340c52a80809>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predicted_y=torch.tensor(predicted_y,dtype=torch.float)
<ipython-input-30-340c52a80809>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.c



Epoch: 8 --> Accuracy 0.12438582880786139




<ipython-input-30-340c52a80809>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predicted_y=torch.tensor(predicted_y,dtype=torch.float)
<ipython-input-30-340c52a80809>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y=torch.tensor(y,dtype=torch.float)
<ipython-input-30-340c52a80809>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predicted_y=torch.tensor(predicted_y,dtype=torch.float)
<ipython-input-30-340c52a80809>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.c



Epoch: 9 --> Accuracy 0.12438582880786139




<ipython-input-30-340c52a80809>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predicted_y=torch.tensor(predicted_y,dtype=torch.float)
<ipython-input-30-340c52a80809>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y=torch.tensor(y,dtype=torch.float)


##Extra

In [ ]:
class MyModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.hidden_size=64
    self.output=1
    self.embedding = nn.Embedding(max_features, embed_size)
    self.rnn = nn.RNN(embed_size, self.hidden_size, batch_first=True)
    self.linear = nn.Linear(self.hidden_size*2 , 64)
    self.sigmoid = nn.Sigmoid()
    self.out=nn.Linear(64,1)

  def forward(self,x):

    input=self.embedding(x)



    input,hn=self.rnn(input)

    avg_pool=torch.mean(input,1)
    max_pool=torch.max(input,1)
    max_values=max_pool.values


    conc = torch.cat((avg_pool, max_values), 1)

    conc=self.linear(conc)

    conc=self.sigmoid(conc)
    out=self.out(conc)

    out=self.sigmoid(out)
    output=torch.tensor(out,dtype=torch.float)

    output.requires_grad=True


    return output



